In [5]:
import requests
from bs4 import BeautifulSoup
import re

homepage = "https://bland.is/messageboard/messageboard.aspx?&page="
domain = "https://bland.is"

thread_links = []

for page in range(1, 10):
    print("Scraping page", page)
    response = requests.get(homepage + str(page))
    soup = BeautifulSoup(response.text, "html.parser")

    # print(soup.prettify())
    # Find all links to threads
    # list_of_threads = soup.find(id="messageboardList")
    # print(thread_links)
    # break
    links = soup.find_all("a", href=re.compile("/umraeda/"))
    links = [domain + link["href"] for link in links]

    thread_links.extend(links)

print(len(thread_links), "threads found")
thread_links[:5]

Scraping page 1
Scraping page 2
Scraping page 3
Scraping page 4
Scraping page 5
Scraping page 6
Scraping page 7
Scraping page 8
Scraping page 9
450 threads found


['https://bland.is/umraeda/veit-einhver-/31483850/?page=1',
 'https://bland.is/umraeda/barnabaekur/31483908/?page=1',
 'https://bland.is/umraeda/leit-af-vinnu-o-/2931707/?page=1',
 'https://bland.is/umraeda/leigubilafyrirtaeki/30548300/?page=1',
 'https://bland.is/umraeda/nytt-fjorflokka-mynstur/31483894/?page=1']

In [6]:
from bs4 import BeautifulSoup, NavigableString, Tag
import json

class Thread:
    def __init__(self, url):
        self.url = url
        self.data = self.get_thread_data(url)

    def __repr__(self):
        return json.dumps(self.data, ensure_ascii=False)

    def clean_text(self, html=""):
        soup = BeautifulSoup(html, 'html.parser')
        text = ''
        for child in soup.find_all('p')[0]:
            if isinstance(child, NavigableString):
                text += str(child).strip()
            elif isinstance(child, Tag):
                if child.name != 'br':
                    text += child.text.strip()
                else:
                    text += '\n'

        # result = text.strip().split('\n')
        result = text.strip()
        return result

    def get_thread_data(self, url):
        response = requests.get(url)
        soup = BeautifulSoup(response.text, "html.parser")
        title = soup.find("h1").text
        question = self.clean_text(str(soup.find("div", class_="firstRecord")))
        responses = soup.find_all("div", class_="mbAnswer")
        # exlude the first response, which is the same as the question
        responses = [self.clean_text(str(response)) for response in responses[1:]]
        return {"title": title.strip(), "question": question, "responses": responses, "url": url}

In [7]:
import tqdm.auto as tqdm
import pandas as pd
import json

with open("bland/data.json", "w") as f:
    f.write("[")

    for url in tqdm.tqdm(thread_links):
        try:
            data = Thread(url).data
            if url != thread_links[0]: f.write(",")
            f.write(json.dumps(data, ensure_ascii=False))

        except Exception as e:
            print(e)
            print("could not scrape:", url)
        
    f.write("]")


  0%|          | 0/450 [00:00<?, ?it/s]

'NoneType' object has no attribute 'text'
could not scrape: https://bland.is/umraeda/oska-eftir-stundirnar-med-gunna-og-felix-a-dvd/31483898/?page=1
'NoneType' object has no attribute 'text'
could not scrape: https://bland.is/umraeda/kratom/31477623/?page=7
list index out of range
could not scrape: https://bland.is/umraeda/baeklunarlaeknar/23980647/?page=8
'NoneType' object has no attribute 'text'
could not scrape: https://bland.is/umraeda/dadur/31481821/?page=9
'NoneType' object has no attribute 'text'
could not scrape: https://bland.is/umraeda/kaupa-fasteign-af-foreldrum/31481792/?page=9


In [8]:
import json
import pandas as pd

with open("bland/data.json", "r") as f:
    data = json.load(f)

df = pd.DataFrame(data)
df.drop(columns=["url", "title"], inplace=True)
df = df[df.responses.apply(len) > 0]

df.head()

with open("bland/cleaned_data.txt", "w") as f:
    for i, row in df.iterrows():
        f.write("###QUESTION:" + "\n" + row.question + "\n\n")
        f.write("###RESPONSE:" + "\n" + row.responses[0] + "\n\n")